Dataset: https://www.kaggle.com/dhruvildave/spotify-charts?select=charts.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Notes: 



*   70 regions.


### Importing necessary libraries.

In [ ]:
!pip3 install spotipy
import pandas as pd
import json
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials

### Loading data.

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Work/ML-Project/data/charts.csv")
df.head()

,title,rank,date,artist,url,region,chart,trend,streams
0,Despacito (Featuring Daddy Yankee),1,2017-03-01,Luis Fonsi,https://open.spotify.com/track/4aWmUDTfIPGksMN...,Argentina,top200,SAME_POSITION,365941.0
1,El Amante,2,2017-03-01,Nicky Jam,https://open.spotify.com/track/3umS4y3uQDkqekN...,Argentina,top200,SAME_POSITION,179697.0
2,Reggaetón Lento (Bailemos),3,2017-03-01,CNCO,https://open.spotify.com/track/3AEZUABDXNtecAO...,Argentina,top200,SAME_POSITION,169647.0
3,Shape of You,4,2017-03-01,Ed Sheeran,https://open.spotify.com/track/7qiZfU4dY1lWllz...,Argentina,top200,SAME_POSITION,168495.0
4,Chantaje (feat. Maluma),5,2017-03-01,Shakira,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,Argentina,top200,SAME_POSITION,141696.0


In [ ]:
len(pd.unique(df['region']))

70

In [ ]:
len(df)

25512202

In [ ]:
len(pd.unique(top200['date'])) # Unique dates Top200.

1762

In [ ]:
top200 = df[df['chart'] == "top200"]
viral50 = df[df['chart'] == 'viral50']

In [ ]:
len(pd.unique(viral50['date'])) # Unique dates Viral50.

1758

In [ ]:
print("Length Top 200:", len(top200))
print("Length Viral 50", len(viral50))

Length Top 200: 19873410
Length Viral 50 5638792


### Cleaning Top200 data

In [ ]:
top200.isnull().sum()

title      7
rank       0
date       0
artist     0
url        0
region     0
chart      0
trend      0
streams    0
dtype: int64

In [ ]:
top200.columns.to_list()

['title',
 'rank',
 'date',
 'artist',
 'url',
 'region',
 'chart',
 'trend',
 'streams']

In [ ]:
top200 = top200[top200['title'].notna()]

In [ ]:
top200.isnull().sum()

title      0
rank       0
date       0
artist     0
url        0
region     0
chart      0
trend      0
streams    0
dtype: int64

### Adding acoustic parameters, popularity, etc to Top200.

In [ ]:
top200['id'] = top200['url'].apply(lambda x: x.split('/')[4])
top200.drop('url',axis=1,inplace=True)

In [ ]:
client_id = '7ac5c16c450e4d59bd2e9cb28dd6f81b'
client_secret = '3811b847b45d4b6282ecc6a8a11878ad'

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [ ]:
song_meta={'id':[],'album':[], 'artist':[], 'explicit':[], 'popularity':[]}

for song_id in top200['id']:
    # get song's meta data
    meta = sp.track(song_id)
    song_meta['id'].append(song_id)
    album = meta['album']['name']
    song_meta['album']+=[album]
    # song name
    #song=meta['name']
    #song_meta['name']+=[song]
    
    # artists name
    s = ', '
    artist=s.join([singer_name['name'] for singer_name in meta['artists']])
    song_meta['artist']+=[artist]
    
    # explicit: lyrics could be considered offensive or unsuitable for children
    explicit=meta['explicit']
    song_meta['explicit'].append(explicit)
    
    # song popularity
    popularity=meta['popularity']
    song_meta['popularity'].append(popularity)

# check the song feature
features = sp.audio_features(song_meta['id'])
# change dictionary to dataframe
features_df = pd.DataFrame.from_dict(features)

# convert milliseconds to mins
# duration_ms: The duration of the track in milliseconds.
# 1 minute = 60 seconds = 60 × 1000 milliseconds = 60,000 ms
features_df['duration_ms']=features_df['duration_ms']/60000
song_meta_df = pd.DataFrame.from_dict(song_meta)

# combine two dataframe
final_df = song_meta_df.merge(features_df)
top200_df = final_df.merge(top200)
top200_df = top200_df.drop(['uri', 'track_href', 'analysis_url'], axis=1)
top200_df = top200_df.sort_values(["date", "rank"], ascending = (True, True))

In [ ]:
top200_df.head()

In [ ]:
top200_df.columns.to_list()

['id',
 'album',
 'artist',
 'explicit',
 'popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'type',
 'uri',
 'track_href',
 'analysis_url',
 'duration_ms',
 'time_signature',
 'title',
 'rank',
 'date',
 'region',
 'chart',
 'trend',
 'streams']

Write to .csv file.

In [ ]:
top200_df.to_csv('/content/top200.csv')